In [4]:
from scipy.stats import chi2_contingency
import pandas as pd
from MakeCytoScapeCSV import EdgesToEndVarEdgeList, ComputeMutualInformation
import numpy as np
from temp import pnummap
import pickle
import random

In [8]:
sev = "If a screening test for SARS-CoV-2 by PCR was performed, what is the most severe severity level (according to WHO) achieved?"
long = "Long Covid"

#sev_list = EdgesToEndVarEdgeList( "Results/AcceptedResults/sev_ranking.txt", sev )
long_list = EdgesToEndVarEdgeList( "Results/AcceptedResults/new_long_cov_ranking.txt", long )

In [9]:
#sev_top = sev_list[ 0:10 ]
long_top = long_list[ 0:10 ]

In [10]:
long_top

['Shortness of breath (Dyspnea) ?',
 'Joint pain (Arthralgia) ?',
 'Loss of taste / lost of smell ?',
 'Chest pain ?',
 'Loss of appetite ?',
 'Fatigue ?',
 'Diarrhea ?',
 'Extremity weakness or numbness ?',
 'Number of comorbidities: ',
 'Nausea / vomiting ?']

In [11]:
#compute the p-value from wilcoxon

def ComputeWilcoxon_pvalue( df, var, end_var ):
    array = pnummap( df, var, end_var )
    #w1, w2 = MakeWilcoxArrays( array )
    #return fisher_exact( [ w1, w2 ] ).pvalue
    return chi2_contingency( array )[ 1 ]

def MakeWilcoxArrays( p_matrix ):
    if ( len(p_matrix) <= 2 ):
        return p_matrix[ 0 ], p_matrix[ 1 ]
    splitter = int(len( p_matrix )/2)
    if ( len(p_matrix) % 2 == 0 ):
        sub_m1 = p_matrix[0: splitter]
        sub_m2 = p_matrix[splitter:]
    else:
        sub_m1 = p_matrix[0: splitter]
        sub_m2 = p_matrix[splitter+1: ]
    return sub_m1.flatten(), sub_m2.flatten()

In [12]:
def GetRandomVarsNotInNetwork( dataframe, network_nodes ):
    vars_not_in = GetVarsNotInNetwork( dataframe, network_nodes )
    randoms = random.sample( vars_not_in, 10)
    return randoms

def GetVarsNotInNetwork( dataframe, network_nodes ):
    not_in = []
    variables = list( dataframe.columns )
    for var in variables:
        if ( var not in network_nodes and var != "Severity" and var != "Long Covid" ):
            not_in.append( var )
    return not_in 

In [13]:
dataframe = pd.read_csv( "CsvData/long.csv" )

p_values_long_best = []
for var in long_top:
    p_values_long_best.append( ComputeWilcoxon_pvalue( dataframe, var, "Long Covid" ) )

In [14]:
file = open( "p_values/new_long_cov_p_values.txt", "w" )
for i in range( len( p_values_long_best) ):
    file.write( long_list[ i ] + ": " + str( p_values_long_best[ i ] ) + "\n")
file.close()
    

In [15]:
sucesses = []
for i in range( 100 ):
    random_vars = GetRandomVarsNotInNetwork( dataframe, long_list )
    p_values = []
    for var in random_vars:
        p_values.append( ComputeWilcoxon_pvalue( dataframe, var, "Long Covid" ) )
    success_count = 0
    for i in range( len( p_values ) ):
        if ( p_values[ i ] < 0.01 ):
            success_count += 1
    sucesses.append( success_count )

In [16]:
print(sucesses)

[3, 2, 3, 2, 2, 1, 4, 3, 2, 1, 2, 1, 1, 1, 1, 2, 3, 3, 2, 2, 2, 0, 1, 1, 1, 2, 1, 3, 2, 2, 4, 3, 0, 2, 2, 0, 1, 2, 2, 3, 2, 3, 1, 3, 2, 0, 2, 2, 0, 1, 2, 4, 1, 3, 3, 1, 2, 0, 3, 1, 2, 1, 2, 3, 1, 2, 0, 1, 0, 1, 0, 2, 1, 1, 2, 2, 2, 2, 2, 0, 2, 3, 2, 3, 2, 2, 2, 1, 3, 2, 4, 0, 3, 2, 2, 1, 2, 1, 1, 2]


In [18]:
print(np.array(sucesses).mean())

1.78
